In [6]:
from particles import datasets as dts
import numpy as np
from scipy.stats import t, norm
import jax.numpy as jnp
import jax.scipy.optimize as opt
import numpy as np
import pickle
from scipy import stats
from scipy import linalg
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import particles.datasets as dts
from particles import resampling as rs
from datetime import datetime
from scipy.stats import qmc



dataset = dts.Pima()
preds = dataset.data
n_preds, dim_max = preds.shape
n_preds = 10
scale_prior = 5 
np.random.seed(10)
dim = 8

beta0 = jnp.zeros(dim)
# noise_level = [10,20,30,40]
# sample_sizes = [2**9,2**10,2**11,2**12,2**13,2**14,2**15,2**16,2**17,2**18]

R = 20 

def minuslogpost(beta, dim, n_preds):
    loglik = jnp.sum( - jnp.log(1. + jnp.exp(-jnp.dot(preds[:n_preds, :dim], beta))))
    logprior = -(0.5 / scale_prior**2) * jnp.sum(beta**2)
    return -(logprior + loglik)

def prior(x):
    return 1 / scale_prior*np.sqrt(2*np.pi) * np.exp(-(0.5 / scale_prior**2) * jnp.sum(x**2))

def norm_density(x):
    return norm.pdf(x)


R = 2 ** 5
Ns = (1021, 2053, 4099, 8191, 16381, 32771, 65537, 131071, 262147,
    524287)


rez = opt.minimize(minuslogpost, beta0, args=(dim,n_preds),
                       method='BFGS', options={'maxiter': 100})
mu = rez.x
Sigma = rez.hess_inv
Cu = linalg.cholesky(Sigma, lower=False)

fl1 = lambda x: np.linalg.det(Cu)*np.exp(jnp.sum( - jnp.log(1. + jnp.exp(-jnp.dot(preds[:n_preds, :dim], x)))))*prior(mu+Cu*x)/norm_density(x)#lapis
fl2 = lambda x: np.exp(jnp.sum( - jnp.log(1. + jnp.exp(-jnp.dot(preds[:n_preds, :dim], x)))))*prior(mu+x)/norm_density(x)#odis
fl3 = lambda x: np.exp(jnp.sum( - jnp.log(1. + jnp.exp(-jnp.dot(preds[:n_preds, :dim], x)))))*prior(x)#none


MCLapIS_est = np.zeros(len(Ns))
MCLapIS_std = np.zeros(len(Ns))
MCODIS_est = np.zeros(len(Ns))
MCODIS_std = np.zeros(len(Ns))
MCNone_est = np.zeros(len(Ns))
MCNone_std = np.zeros(len(Ns))
RQMCLapIS_est = np.zeros(len(Ns))
RQMCLapIS_std = np.zeros(len(Ns))
RQMCODIS_est = np.zeros(len(Ns))
RQMCODIS_std = np.zeros(len(Ns))
RQMCNone_est = np.zeros(len(Ns))
RQMCNone_std = np.zeros(len(Ns))

for i, N in enumerate(Ns):
    print(f'N={N}', datetime.now())
    generator_z = np.load(
        f'./generators_lattice_rules/generator{N}.npy')[:dim]
    latticerule = generator_z * (np.arange(N).reshape((N, 1))/N)
    MC_means1 = np.zeros(R)
    MC_means2 = np.zeros(R)
    MC_means3 = np.zeros(R)
    
    RQMC_means1 = np.zeros(R)
    RQMC_means2 = np.zeros(R)
    RQMC_means3 = np.zeros(R)
    for r in range(R):
        # MC
        X = norm.rvs(size=(N, dim))
        MC_means1[r] = np.mean(list(map(fl1, X)))
        MC_means2[r] = np.mean(list(map(fl2, X)))
        MC_means3[r] = np.mean(list(map(fl3, X)))
        # RQMC
        shift = np.random.sample(size=dim)
        X = norm.ppf(np.modf(latticerule+shift)[0])
        RQMC_means1[r] = np.mean(list(map(fl1, X)))
        RQMC_means2[r] = np.mean(list(map(fl2, X)))
        RQMC_means3[r] = np.mean(list(map(fl3, X)))
        if r+1 in [4*i for i in range(1, R//4)]:
            print(f'N={N}, {r+1}th iteration,', datetime.now())
    
    MCLapIS_est[i] = np.mean(MC_means1)
    MCLapIS_std[i] = np.std(MC_means1, ddof=1)
    MCODIS_est[i] = np.mean(MC_means2)
    MCODIS_std[i] = np.std(MC_means2, ddof=1)
    MCNone_est[i] = np.mean(MC_means3)
    MCNone_std[i] = np.std(MC_means3, ddof=1)
    RQMCLapIS_est[i] = np.mean(RQMC_means1)
    RQMCLapIS_std[i] = np.std(RQMC_means1, ddof=1)
    RQMCODIS_est[i] = np.mean(RQMC_means2)
    RQMCODIS_std[i] = np.std(RQMC_means2, ddof=1)
    RQMCNone_est[i] = np.mean(RQMC_means3)
    RQMCNone_std[i] = np.std(RQMC_means3, ddof=1)

    print(f'estimated mean and std for N={N}:')

    print(f'MC: {MCLapIS_est[i]:.8f} {MCLapIS_std[i]:.2e}')
    print(f'MC: {MCODIS_est[i]:.8f} {MCODIS_std[i]:.2e}')
    print(f'MC: {MCNone_est[i]:.8f} {MCNone_std[i]:.2e}')
    print(f'RQMC: {RQMCLapIS_est[i]:.8f} {RQMCLapIS_std[i]:.2e}')
    print(f'RQMC: {RQMCODIS_est[i]:.8f} {RQMCODIS_std[i]:.2e}')
    print(f'RQMC: {RQMCNone_est[i]:.8f} {RQMCNone_std[i]:.2e}')
    print(datetime.now())
    print()

np.save('./8dimMCLapIS_est.npy', MCLapIS_est)
np.save('./8dimMCLapIS_std.npy', MCLapIS_std)
np.save('./8dimMCODIS_est.npy', MCODIS_est)
np.save('./8dimMCODIS_std.npy', MCODIS_std)
np.save('./8dimMCNone_est.npy', MCNone_est)
np.save('./8dimMCNone_std.npy', MCNone_std)
np.save('./8dimRQMCLapIS_est.npy', RQMCLapIS_est)
np.save('./8dimRQMCLapIS_std.npy', RQMCLapIS_std)
np.save('./8dimRQMCODIS_est.npy', RQMCODIS_est)
np.save('./8dimRQMCODIS_std.npy', RQMCODIS_std)
np.save('./8dimRQMCNone_est.npy', RQMCNone_est)
np.save('./8dimRQMCNone_std.npy', RQMCNone_std)







N=1021 2024-01-25 22:51:42.916121
N=1021, 4th iteration, 2024-01-25 22:51:45.224269
N=1021, 8th iteration, 2024-01-25 22:51:47.450049
N=1021, 12th iteration, 2024-01-25 22:51:49.632127
N=1021, 16th iteration, 2024-01-25 22:51:51.804570
N=1021, 20th iteration, 2024-01-25 22:51:53.983667
N=1021, 24th iteration, 2024-01-25 22:51:56.170586
N=1021, 28th iteration, 2024-01-25 22:51:58.344702
estimated mean and std for N=1021:
MC: 0.00016708 1.32e-05
MC: 0.07916634 9.74e-03
MC: 0.00033685 3.33e-05
RQMC: 0.00017109 1.78e-05
RQMC: 0.12563534 2.67e-01
RQMC: 0.00034839 2.14e-05
2024-01-25 22:52:00.515951

N=2053 2024-01-25 22:52:00.515961
N=2053, 4th iteration, 2024-01-25 22:52:04.874935
N=2053, 8th iteration, 2024-01-25 22:52:09.258038
N=2053, 12th iteration, 2024-01-25 22:52:13.587221
N=2053, 16th iteration, 2024-01-25 22:52:17.939043
N=2053, 20th iteration, 2024-01-25 22:52:22.296607
N=2053, 24th iteration, 2024-01-25 22:52:26.652814


KeyboardInterrupt: 

In [ ]:
from particles import datasets as dts
import numpy as np
from scipy.stats import t, norm
import jax.numpy as jnp
import jax.scipy.optimize as opt
import numpy as np
import pickle
from scipy import stats
from scipy import linalg
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import particles.datasets as dts
from particles import resampling as rs
from datetime import datetime
from scipy.stats import qmc



dataset = dts.Pima()
preds = dataset.data
n_preds, dim_max = preds.shape
n_preds = 100
scale_prior = 5 
np.random.seed(10)
dim = 8

beta0 = jnp.zeros(dim)
# noise_level = [10,20,30,40]
# sample_sizes = [2**9,2**10,2**11,2**12,2**13,2**14,2**15,2**16,2**17,2**18]

R = 20 

def minuslogpost(beta, dim, n_preds):
    loglik = jnp.sum( - jnp.log(1. + jnp.exp(-jnp.dot(preds[:n_preds, :dim], beta))))
    logprior = -(0.5 / scale_prior**2) * jnp.sum(beta**2)
    return -(logprior + loglik)

def prior(x):
    return 1 / scale_prior*np.sqrt(2*np.pi) * np.exp(-(0.5 / scale_prior**2) * jnp.sum(x**2))

def norm_density(x):
    return norm.pdf(x)


R = 2 ** 6
Ns = (1021, 2053, 4099, 8191, 16381, 32771, 65537, 131071, 262147,
    524287)


rez = opt.minimize(minuslogpost, beta0, args=(dim,n_preds),
                       method='BFGS', options={'maxiter': 100})
mu = rez.x
Sigma = rez.hess_inv
Cu = linalg.cholesky(Sigma, lower=False)

fl1 = lambda x: np.exp(jnp.sum( - jnp.log(1. + jnp.exp(-jnp.dot(preds[:n_preds, :dim], mu+x)))))*prior(mu+x)/norm_density(x)


MC_est = np.zeros(len(Ns))
MC_std = np.zeros(len(Ns))
RQMC_est = np.zeros(len(Ns))
RQMC_std = np.zeros(len(Ns))

for i, N in enumerate(Ns):
    print(f'N={N}', datetime.now())
    generator_z = np.load(
        f'./generators_lattice_rules/generator{N}.npy')[:dim]
    latticerule = generator_z * (np.arange(N).reshape((N, 1))/N)
    MC_means = np.zeros(R)
    RQMC_means = np.zeros(R)
    
    for r in range(R):
        # MC
        X = norm.rvs(size=(N, dim))
        MC_means[r] = np.mean(list(map(fl1, X)))
        # RQMC
        shift = np.random.sample(size=dim)
        X = norm.ppf(np.modf(latticerule+shift)[0])
        RQMC_means[r] = np.mean(list(map(fl1, X)))
        if r+1 in [4*i for i in range(1, R//4)]:
            print(f'N={N}, {r+1}th iteration,', datetime.now())



    MC_est[i] = np.mean(MC_means)
    MC_std[i] = np.std(MC_means, ddof=1)
    RQMC_est[i] = np.mean(RQMC_means)
    RQMC_std[i] = np.std(RQMC_means, ddof=1)
    print(f'estimated mean and std for N={N}:')
    print(f'MC: {MC_est[i]:.5f} {MC_std[i]:.2e}')
    print(f'RQMC: {RQMC_est[i]:.5f} {RQMC_std[i]:.2e}')
    print(datetime.now())
    print()


np.save('./8dimMCODIS_est.npy', MC_est)
np.save('./8dimMCODIS_std.npy', MC_std)
np.save('./8dimRQMCODIS_est.npy', RQMC_est)
np.save('./8dimRQMCODIS_std.npy', RQMC_std)